# Connecting to SQLite Database and Analyzing

In [1]:
import pandas as pd
import numpy as np
import csv
import sqlite3



db_path = "xeneta.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()



cursor.execute("SELECT name FROM sqlite_master WHERE type='table';") # getting table name from Database
tables = cursor.fetchall()
print("Tables in the database:", tables)




Tables in the database: [('ports',)]


In [2]:
#We convert the database into CSV format to observe the data more transparently. 

for table in tables:
    ports = table[0]
    cursor.execute(f"SELECT * FROM ports;")
    rows = cursor.fetchall()

    # Save to CSV
    with open(f"ports.csv", "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerows(rows)
        


In [3]:

# Query to fetch data from the 'ports' table
cursor.execute("SELECT * FROM ports;")
ports_data = cursor.fetchall()  # Fetch all the rows

# load the data into "lookups" DataFrame for getting the port name and port code from xeneta.db file

ports_columns = [description[0] for description in cursor.description]  # Get column names
lookups = pd.DataFrame(ports_data, columns=ports_columns)

# Verify the loaded lookups DataFrame
print(lookups.columns)
print(lookups.head())  # show the first few rows of data

# Close the database connection
conn.close()




'''
#############Assumption##################
It is assumed that the `ports` table in the database is complete, accurately populated with all required port names and codes, and structured with columns named `port_name` and `port_code`. 
Additionally, the database connection is functional, and the data retrieved matches the expected format without missing or inconsistent entries.

'''

Index(['port_name', 'port_code'], dtype='object')
               port_name port_code
0          Anqing, China     CNAQG
1    Arkhangelsk, Russia     RUARH
2      Bangkok, Thailand     THBKK
3  Batangas, Philippines     PHBTG
4          Beihai, China     CNBIH


'\n#############Assumption##################\nIt is assumed that the `ports` table in the database is complete, accurately populated with all required port names and codes, and structured with columns named `port_name` and `port_code`. \nAdditionally, the database connection is functional, and the data retrieved matches the expected format without missing or inconsistent entries.\n\n'

# Processed the data to convert them into "Final Import" sheet format

In [4]:
# Define the Excel file and sheet names
excel_path = "data_science_test_1.xlsx"
sheet_name = "Ocean Freight Rates"
final_import_sheet_name = "Final Import"
lookups_sheet_name = "Lookups"


ocean_rates = pd.read_excel(excel_path, sheet_name=sheet_name, skiprows=6) # Read the actual data starting from row 7, thats why we use skiprows=6
final_import = pd.read_excel(excel_path, sheet_name=final_import_sheet_name)
look_ups = pd.read_excel(excel_path, sheet_name=lookups_sheet_name)

In [5]:
#Checking if "lookups" (from xeneta.db) and "look_ups" (excel file data: Lookups sheet) are the same or not!

print(lookups)
lookups.info()
print(look_ups)

look_ups.info()



'''
#############Assumption##################
It is assumed that the `lookups` DataFrame is correctly loaded from the database and contains two columns: `port_name` and `port_code`, with 147 non-null entries for each. 
The data in these columns is assumed to be in the expected format, with valid port names and corresponding port codes. 
Additionally, it is assumed that the DataFrame is properly structured, and no data inconsistencies or missing values exist in the `port_name` or `port_code` columns.
'''

                 port_name port_code
0            Anqing, China     CNAQG
1      Arkhangelsk, Russia     RUARH
2        Bangkok, Thailand     THBKK
3    Batangas, Philippines     PHBTG
4            Beihai, China     CNBIH
..                     ...       ...
142       Zhanjiang, China     CNZNG
143        Zhaoqing, China     CNZQG
144       Zhenjiang, China     CNZHE
145       Zhongshan, China     CNZSN
146          Zhuhai, China     CNZUH

[147 rows x 2 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147 entries, 0 to 146
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   port_name  147 non-null    object
 1   port_code  147 non-null    object
dtypes: object(2)
memory usage: 2.4+ KB
                 port_name port_code
0            Anqing, China     CNAQG
1      Arkhangelsk, Russia     RUARH
2        Bangkok, Thailand     THBKK
3    Batangas, Philippines     PHBTG
4            Beihai, China     CNBIH
..        

'\n#############Assumption##################\nIt is assumed that the `lookups` DataFrame is correctly loaded from the database and contains two columns: `port_name` and `port_code`, with 147 non-null entries for each. \nThe data in these columns is assumed to be in the expected format, with valid port names and corresponding port codes. \nAdditionally, it is assumed that the DataFrame is properly structured, and no data inconsistencies or missing values exist in the `port_name` or `port_code` columns.\n'

In [6]:
'''
#############Assumption##################
The "Ocean Freight Rates" sheet contains some metadata (headers) such as customer name, customer contract number, and carrier.  
It is necessary to extract this information and organize it into a structured, tabular format.  
We extract the metadata from the "Ocean Freight Rates" sheet using a key-value approach and place it into a table.  

'''


# Function to extract metadata from the Excel file
def extract_metadata_from_excel(excel_path, sheet_name, keywords):
    metadata = {}
   
    df = pd.read_excel(excel_path, sheet_name=sheet_name, nrows=10)   
    
    # iterate through each row and extract metadata until a blank cell or line break is encountered
    for index, row in df.iterrows():
        for keyword in keywords:
            if any(keyword in str(cell) for cell in row):
                # extract metadata as long as there's no blank cell or line break
                metadata[keyword] = None
                for col in row.dropna():
                    # check if the cell contains a line break or is blank
                    if isinstance(col, str) and ('\n' in col or col.strip() == ''):
                        break
                    metadata[keyword] = col  # store the value
    return metadata


# Define keywords for metadata extraction
keywords = ['Customer', 'Contract Number', 'Carrier']

# Extract metadata from the Excel file
metadata = extract_metadata_from_excel(excel_path, sheet_name, keywords)
metadata

{'Customer': 'AWESOME CUSTOMER INC.',
 'Contract Number': 'Super Secret Contract',
 'Carrier': 'XENETA'}

In [7]:

# Add metadata to the DataFrame for all rows
for key, value in metadata.items():
    ocean_rates[key] = value
    
ocean_rates.info() #now we will able to see that the metadata info also showing in the ocean_rates dataframe as column

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16654 entries, 0 to 16653
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Receipt               16653 non-null  object 
 1   Load Port             121 non-null    object 
 2   Discharge Port        0 non-null      float64
 3   Delivery              16652 non-null  object 
 4   Effective Date        16654 non-null  object 
 5   Expiry Date           16654 non-null  object 
 6   Service Mode          16654 non-null  object 
 7   Commodity Name        16654 non-null  object 
 8   Inclusive Surcharges  16654 non-null  object 
 9   Charge                16654 non-null  object 
 10  Rate Basis            16654 non-null  object 
 11  20DRY                 16571 non-null  object 
 12  40DRY                 16653 non-null  object 
 13  40HDRY                16646 non-null  object 
 14  45HDRY                15032 non-null  object 
 15  Customer           

In [8]:

'''
#############Assumption##################
It is assumed that the `ocean_rates` DataFrame contains valid data for columns such as "Commodity Name," "Service Mode," "Rate Basis," and "Equipment Type." 
We include relevant information for filtering based on the specified criteria (e.g., "FAK" for Commodity Name, "CY/CY" for Service Mode, and "PER_CONTAINER" for Rate Basis). 
Additionally, it is assumed that the equipment types in the dataset are represented as `20DRY`, `40DRY`, `40HDRY`, and `45HDRY`, and that these need to be reshaped and renamed accordingly, with `45HDRY` being excluded from the analysis.

'''


# Filter data for FAK Commodity and CY/CY Service Mode
ocean_rates = ocean_rates[(ocean_rates["Commodity Name"] == "FAK") & (ocean_rates["Service Mode"] == "CY/CY")]

# Filter for PER_CONTAINER Rate Basis
ocean_rates = ocean_rates[ocean_rates["Rate Basis"] == "PER_CONTAINER"]

# Reshape Equipment Type Columns
equipment_columns = ['20DRY', '40DRY', '40HDRY', '45HDRY']
ocean_rates = ocean_rates.melt(
    id_vars=['Receipt', 'Load Port', 'Discharge Port', 'Delivery', 'Effective Date',
             'Expiry Date', 'Service Mode', 'Commodity Name', 'Inclusive Surcharges',
             'Charge', 'Rate Basis', 'Customer', 'Contract Number', 'Carrier'],
    value_vars=equipment_columns,
    var_name='Equipment Type',
    value_name='Rate'
)

# Exclude rows with 45HDRY equipment type
ocean_rates = ocean_rates[ocean_rates['Equipment Type'] != '45HDRY']

# Rename Equipment Types
equipment_mapping = {
    "20DRY": "20DC",
    "40DRY": "40DC",
    "40HDRY": "40HC"
}
ocean_rates['Equipment Type'] = ocean_rates['Equipment Type'].map(equipment_mapping)
ocean_rates

,Receipt,Load Port,Discharge Port,Delivery,Effective Date,Expiry Date,Service Mode,Commodity Name,Inclusive Surcharges,Charge,Rate Basis,Customer,Contract Number,Carrier,Equipment Type,Rate
0,"Anqing, China",NaN,NaN,"Cochin, India",29-Apr-2011,12-May-2011,CY/CY,FAK,"DPA,OPA,SBF,BAF",BAS,PER_CONTAINER,AWESOME CUSTOMER INC.,Super Secret Contract,XENETA,20DC,760 USD
1,"Anqing, China",NaN,NaN,"Cochin, India",29-Apr-2011,12-May-2011,CY/CY,FAK,"DPA,OPA,SBF,BAF",CFD,PER_CONTAINER,AWESOME CUSTOMER INC.,Super Secret Contract,XENETA,20DC,0 USD
2,"Anqing, China",NaN,NaN,"Cochin, India",29-Apr-2011,12-May-2011,CY/CY,FAK,"DPA,OPA,SBF,BAF",CFO,PER_CONTAINER,AWESOME CUSTOMER INC.,Super Secret Contract,XENETA,20DC,0 USD
3,"Anqing, China",NaN,NaN,"Cochin, India",29-Apr-2011,12-May-2011,CY/CY,FAK,"DPA,OPA,SBF,BAF",DHC,PER_CONTAINER,AWESOME CUSTOMER INC.,Super Secret Contract,XENETA,20DC,10175 INR
4,"Anqing, China",NaN,NaN,"Cochin, India",29-Apr-2011,12-May-2011,CY/CY,FAK,"DPA,OPA,SBF,BAF",ERS,PER_CONTAINER,AWESOME CUSTOMER INC.,Super Secret Contract,XENETA,20DC,0 USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41548,"Zhuhai, China",NaN,NaN,"Visakhapatnam, India",29-Apr-2011,12-May-2011,CY/CY,FAK,"DPA,OPA,SBF,BAF",EXP,PER_CONTAINER,AWESOME CUSTOMER INC.,Super Secret Contract,XENETA,40HC,50 CNY
41549,"Zhuhai, China",NaN,NaN,"Visakhapatnam, India",29-Apr-2011,12-May-2011,CY/CY,FAK,"DPA,OPA,SBF,BAF",IMP,PER_CONTAINER,AWESOME CUSTOMER INC.,Super Secret Contract,XENETA,40HC,2600 INR
41550,"Zhuhai, China",NaN,NaN,"Visakhapatnam, India",29-Apr-2011,12-May-2011,CY/CY,FAK,"DPA,OPA,SBF,BAF",LSS,PER_CONTAINER,AWESOME CUSTOMER INC.,Super Secret Contract,XENETA,40HC,24 USD
41551,"Zhuhai, China",NaN,NaN,"Visakhapatnam, India",29-Apr-2011,12-May-2011,CY/CY,FAK,"DPA,OPA,SBF,BAF",OHC,PER_CONTAINER,AWESOME CUSTOMER INC.,Super Secret Contract,XENETA,40HC,890 CNY


In [9]:
'''
#############Assumption##################
It is assumed that the `ocean_rates` DataFrame contains valid and consistent data for columns such as "Inclusive Surcharges" and "Rate."
It is assumed that the presence of "OHC" or "DHC" values in the "Inclusive Surcharges" column, or a rate of 0, correctly indicates whether the respective surcharges should be classified as "OTHC," "DTHC," "BOTH," or "NONE."
The logic for these classifications is expected to be applied to all rows, with the "THC Used" value being assigned accordingly.

'''

# Calculate THC Used

def calculate_thc_used(row):
    inclusive = row["Inclusive Surcharges"] or ""
    rate = row["Rate"]
    
    # Check for OHC and DHC presence more robustly
    if "OHC" in inclusive or (rate == 0 and "OHC" not in inclusive):
        ohc_inclusive = True
    else:
        ohc_inclusive = False

    if "DHC" in inclusive or (rate == 0 and "DHC" not in inclusive):
        dhc_inclusive = True
    else:
        dhc_inclusive = False

    # Determine THC Used based on the surcharges
    if ohc_inclusive and dhc_inclusive:
        return "BOTH"
    elif ohc_inclusive:
        return "OTHC"
    elif dhc_inclusive:
        return "DTHC"
    else:
        return "NONE"


ocean_rates["THC Used"] = ocean_rates.apply(calculate_thc_used, axis=1)




ocean_rates

,Receipt,Load Port,Discharge Port,Delivery,Effective Date,Expiry Date,Service Mode,Commodity Name,Inclusive Surcharges,Charge,Rate Basis,Customer,Contract Number,Carrier,Equipment Type,Rate,THC Used
0,"Anqing, China",NaN,NaN,"Cochin, India",29-Apr-2011,12-May-2011,CY/CY,FAK,"DPA,OPA,SBF,BAF",BAS,PER_CONTAINER,AWESOME CUSTOMER INC.,Super Secret Contract,XENETA,20DC,760 USD,NONE
1,"Anqing, China",NaN,NaN,"Cochin, India",29-Apr-2011,12-May-2011,CY/CY,FAK,"DPA,OPA,SBF,BAF",CFD,PER_CONTAINER,AWESOME CUSTOMER INC.,Super Secret Contract,XENETA,20DC,0 USD,NONE
2,"Anqing, China",NaN,NaN,"Cochin, India",29-Apr-2011,12-May-2011,CY/CY,FAK,"DPA,OPA,SBF,BAF",CFO,PER_CONTAINER,AWESOME CUSTOMER INC.,Super Secret Contract,XENETA,20DC,0 USD,NONE
3,"Anqing, China",NaN,NaN,"Cochin, India",29-Apr-2011,12-May-2011,CY/CY,FAK,"DPA,OPA,SBF,BAF",DHC,PER_CONTAINER,AWESOME CUSTOMER INC.,Super Secret Contract,XENETA,20DC,10175 INR,NONE
4,"Anqing, China",NaN,NaN,"Cochin, India",29-Apr-2011,12-May-2011,CY/CY,FAK,"DPA,OPA,SBF,BAF",ERS,PER_CONTAINER,AWESOME CUSTOMER INC.,Super Secret Contract,XENETA,20DC,0 USD,NONE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41548,"Zhuhai, China",NaN,NaN,"Visakhapatnam, India",29-Apr-2011,12-May-2011,CY/CY,FAK,"DPA,OPA,SBF,BAF",EXP,PER_CONTAINER,AWESOME CUSTOMER INC.,Super Secret Contract,XENETA,40HC,50 CNY,NONE
41549,"Zhuhai, China",NaN,NaN,"Visakhapatnam, India",29-Apr-2011,12-May-2011,CY/CY,FAK,"DPA,OPA,SBF,BAF",IMP,PER_CONTAINER,AWESOME CUSTOMER INC.,Super Secret Contract,XENETA,40HC,2600 INR,NONE
41550,"Zhuhai, China",NaN,NaN,"Visakhapatnam, India",29-Apr-2011,12-May-2011,CY/CY,FAK,"DPA,OPA,SBF,BAF",LSS,PER_CONTAINER,AWESOME CUSTOMER INC.,Super Secret Contract,XENETA,40HC,24 USD,NONE
41551,"Zhuhai, China",NaN,NaN,"Visakhapatnam, India",29-Apr-2011,12-May-2011,CY/CY,FAK,"DPA,OPA,SBF,BAF",OHC,PER_CONTAINER,AWESOME CUSTOMER INC.,Super Secret Contract,XENETA,40HC,890 CNY,NONE


In [10]:
# Handle Origin and Destination Ports
def get_origin_port(row):
    return row["Load Port"] if pd.notna(row["Load Port"]) else row["Receipt"]

def get_destination_port(row):
    return row["Discharge Port"] if pd.notna(row["Discharge Port"]) else row["Delivery"]

ocean_rates["Origin Port"] = ocean_rates.apply(get_origin_port, axis=1)
ocean_rates["Destination Port"] = ocean_rates.apply(get_destination_port, axis=1)

ocean_rates #origin port and destination port added in the ocean_rates dataframe

,Receipt,Load Port,Discharge Port,Delivery,Effective Date,Expiry Date,Service Mode,Commodity Name,Inclusive Surcharges,Charge,Rate Basis,Customer,Contract Number,Carrier,Equipment Type,Rate,THC Used,Origin Port,Destination Port
0,"Anqing, China",NaN,NaN,"Cochin, India",29-Apr-2011,12-May-2011,CY/CY,FAK,"DPA,OPA,SBF,BAF",BAS,PER_CONTAINER,AWESOME CUSTOMER INC.,Super Secret Contract,XENETA,20DC,760 USD,NONE,"Anqing, China","Cochin, India"
1,"Anqing, China",NaN,NaN,"Cochin, India",29-Apr-2011,12-May-2011,CY/CY,FAK,"DPA,OPA,SBF,BAF",CFD,PER_CONTAINER,AWESOME CUSTOMER INC.,Super Secret Contract,XENETA,20DC,0 USD,NONE,"Anqing, China","Cochin, India"
2,"Anqing, China",NaN,NaN,"Cochin, India",29-Apr-2011,12-May-2011,CY/CY,FAK,"DPA,OPA,SBF,BAF",CFO,PER_CONTAINER,AWESOME CUSTOMER INC.,Super Secret Contract,XENETA,20DC,0 USD,NONE,"Anqing, China","Cochin, India"
3,"Anqing, China",NaN,NaN,"Cochin, India",29-Apr-2011,12-May-2011,CY/CY,FAK,"DPA,OPA,SBF,BAF",DHC,PER_CONTAINER,AWESOME CUSTOMER INC.,Super Secret Contract,XENETA,20DC,10175 INR,NONE,"Anqing, China","Cochin, India"
4,"Anqing, China",NaN,NaN,"Cochin, India",29-Apr-2011,12-May-2011,CY/CY,FAK,"DPA,OPA,SBF,BAF",ERS,PER_CONTAINER,AWESOME CUSTOMER INC.,Super Secret Contract,XENETA,20DC,0 USD,NONE,"Anqing, China","Cochin, India"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41548,"Zhuhai, China",NaN,NaN,"Visakhapatnam, India",29-Apr-2011,12-May-2011,CY/CY,FAK,"DPA,OPA,SBF,BAF",EXP,PER_CONTAINER,AWESOME CUSTOMER INC.,Super Secret Contract,XENETA,40HC,50 CNY,NONE,"Zhuhai, China","Visakhapatnam, India"
41549,"Zhuhai, China",NaN,NaN,"Visakhapatnam, India",29-Apr-2011,12-May-2011,CY/CY,FAK,"DPA,OPA,SBF,BAF",IMP,PER_CONTAINER,AWESOME CUSTOMER INC.,Super Secret Contract,XENETA,40HC,2600 INR,NONE,"Zhuhai, China","Visakhapatnam, India"
41550,"Zhuhai, China",NaN,NaN,"Visakhapatnam, India",29-Apr-2011,12-May-2011,CY/CY,FAK,"DPA,OPA,SBF,BAF",LSS,PER_CONTAINER,AWESOME CUSTOMER INC.,Super Secret Contract,XENETA,40HC,24 USD,NONE,"Zhuhai, China","Visakhapatnam, India"
41551,"Zhuhai, China",NaN,NaN,"Visakhapatnam, India",29-Apr-2011,12-May-2011,CY/CY,FAK,"DPA,OPA,SBF,BAF",OHC,PER_CONTAINER,AWESOME CUSTOMER INC.,Super Secret Contract,XENETA,40HC,890 CNY,NONE,"Zhuhai, China","Visakhapatnam, India"


In [11]:
# Match Port Codes


use_sqlite = True
if use_sqlite:
    port_lookup = lookups
else:
    port_lookup = look_ups

# Create a mapping of port names to port codes for lookups
port_mapping = dict(zip(port_lookup["port_name"], port_lookup["port_code"]))

# Match Origin and Destination port codes based on the normalized names
ocean_rates["Origin Port Code"] = ocean_rates["Origin Port"].map(port_mapping)
ocean_rates["Destination Port Code"] = ocean_rates["Destination Port"].map(port_mapping)

ocean_rates

'''
#############Assumption##################
Now we can see the port name with port code in the dataframe with matched port codes
'''

'\n#############Assumption##################\nNow we can see the port name with port code in the dataframe with matched port codes\n'

In [12]:


# Replace 0 values with NaN
ocean_rates = ocean_rates.replace(0, np.nan)

# Identify surcharge columns (assuming these are the columns that represent surcharges)
surcharge_columns = [col for col in ocean_rates.columns if 'Charge' in col or 'Surcharge' in col]

# For each surcharge column, check if there is any non-NaN (non-zero) value, and drop those that contain only NaNs
for column in surcharge_columns:
    if ocean_rates[column].isna().all(): 
        ocean_rates = ocean_rates.drop(columns=[column])  

# print out the columns that have been dropped for transparency (if any actually)
print("Dropped surcharge columns due to all zero values:", [col for col in surcharge_columns if ocean_rates[column].isna().all()])



Dropped surcharge columns due to all zero values: []


In [13]:
# Function to extract currency from 'Rate' column

def extract_currency(rate_value):
    if isinstance(rate_value, str):
        parts = rate_value.split()  # Split by space and take the last part as the currency
        if len(parts) > 1:
            return parts[-1]  # Return the currency (e.g., USD, EUR)
    return np.nan     # Return NaN if no valid currency is found



# extract Currency from 'Rate' column and assign to the respective columns
ocean_rates['BAS, Currency'] = ocean_rates['Rate'].apply(
    lambda x: extract_currency(x) if pd.notna(x) else np.nan
)

ocean_rates['RHI, Currency'] = ocean_rates['Rate'].apply(
    lambda x: extract_currency(x) if pd.notna(x) else np.nan
)



In [14]:
'''

It is assumed that the `ocean_rates` DataFrame contains a valid "Rate" column and an "Equipment Type" column with values such as '20DC', '40DC', and '40HC'. 
The assumption is that the equipment types present in the data are strictly one of these types, and if an entry does not match, the corresponding rate is assigned as `np.nan`. 
Additionally, it is assumed that the final import column names such as ['BAS, 20DC'], ['BAS, 40DC'], ['BAS, 40HC'], ['RHI, 20DC'], ['RHI, 40DC'], and ['RHI, 40HC'] are the desired column names for rate values corresponding to these equipment types. 
The logic applies the rate value to the appropriate columns based on the equipment type, and for other types, it assigns `np.nan`. 
The data is expected to be accurately structured with the correct "Equipment Type" for each row to ensure proper column assignment.
'''


# Rename Charge columns to match with final import column: ['BAS, 20DC'], ['BAS, 40DC'], ['BAS, 40HC']
ocean_rates['BAS, 20DC'] = ocean_rates.apply(lambda row: row['Rate'] if row['Equipment Type'] == '20DC' else np.nan, axis=1)
ocean_rates['BAS, 40DC'] = ocean_rates.apply(lambda row: row['Rate'] if row['Equipment Type'] == '40DC' else np.nan, axis=1)
ocean_rates['BAS, 40HC'] = ocean_rates.apply(lambda row: row['Rate'] if row['Equipment Type'] == '40HC' else np.nan, axis=1)

# Rename Charge columns to match with final import column: ['RHI, 20DC'], ['RHI, 40DC'], ['RHI, 40HC']
ocean_rates['RHI, 20DC'] = ocean_rates.apply(lambda row: row['Rate'] if row['Equipment Type'] == '20DC' else np.nan, axis=1)
ocean_rates['RHI, 40DC'] = ocean_rates.apply(lambda row: row['Rate'] if row['Equipment Type'] == '40DC' else np.nan, axis=1)
ocean_rates['RHI, 40HC'] = ocean_rates.apply(lambda row: row['Rate'] if row['Equipment Type'] == '40HC' else np.nan, axis=1)


ocean_rates #added "BAS, 20DC"; "BAS, 40DC"; "BAS, 40HC"; "RHI, 20DC"; "RHI, 40DC"; added "RHI, 40HC"


,Receipt,Load Port,Discharge Port,Delivery,Effective Date,Expiry Date,Service Mode,Commodity Name,Inclusive Surcharges,Charge,...,Origin Port Code,Destination Port Code,"BAS, Currency","RHI, Currency","BAS, 20DC","BAS, 40DC","BAS, 40HC","RHI, 20DC","RHI, 40DC","RHI, 40HC"
0,"Anqing, China",NaN,NaN,"Cochin, India",29-Apr-2011,12-May-2011,CY/CY,FAK,"DPA,OPA,SBF,BAF",BAS,...,CNAQG,INCOK,USD,USD,760 USD,NaN,NaN,760 USD,NaN,NaN
1,"Anqing, China",NaN,NaN,"Cochin, India",29-Apr-2011,12-May-2011,CY/CY,FAK,"DPA,OPA,SBF,BAF",CFD,...,CNAQG,INCOK,USD,USD,0 USD,NaN,NaN,0 USD,NaN,NaN
2,"Anqing, China",NaN,NaN,"Cochin, India",29-Apr-2011,12-May-2011,CY/CY,FAK,"DPA,OPA,SBF,BAF",CFO,...,CNAQG,INCOK,USD,USD,0 USD,NaN,NaN,0 USD,NaN,NaN
3,"Anqing, China",NaN,NaN,"Cochin, India",29-Apr-2011,12-May-2011,CY/CY,FAK,"DPA,OPA,SBF,BAF",DHC,...,CNAQG,INCOK,INR,INR,10175 INR,NaN,NaN,10175 INR,NaN,NaN
4,"Anqing, China",NaN,NaN,"Cochin, India",29-Apr-2011,12-May-2011,CY/CY,FAK,"DPA,OPA,SBF,BAF",ERS,...,CNAQG,INCOK,USD,USD,0 USD,NaN,NaN,0 USD,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41548,"Zhuhai, China",NaN,NaN,"Visakhapatnam, India",29-Apr-2011,12-May-2011,CY/CY,FAK,"DPA,OPA,SBF,BAF",EXP,...,CNZUH,INVTZ,CNY,CNY,NaN,NaN,50 CNY,NaN,NaN,50 CNY
41549,"Zhuhai, China",NaN,NaN,"Visakhapatnam, India",29-Apr-2011,12-May-2011,CY/CY,FAK,"DPA,OPA,SBF,BAF",IMP,...,CNZUH,INVTZ,INR,INR,NaN,NaN,2600 INR,NaN,NaN,2600 INR
41550,"Zhuhai, China",NaN,NaN,"Visakhapatnam, India",29-Apr-2011,12-May-2011,CY/CY,FAK,"DPA,OPA,SBF,BAF",LSS,...,CNZUH,INVTZ,USD,USD,NaN,NaN,24 USD,NaN,NaN,24 USD
41551,"Zhuhai, China",NaN,NaN,"Visakhapatnam, India",29-Apr-2011,12-May-2011,CY/CY,FAK,"DPA,OPA,SBF,BAF",OHC,...,CNZUH,INVTZ,CNY,CNY,NaN,NaN,890 CNY,NaN,NaN,890 CNY


In [15]:

# drop unnecessary columns
ocean_rates = ocean_rates.drop(columns=['Rate', 'Equipment Type'])

# remove Duplicates
ocean_rates = ocean_rates.drop_duplicates()


# align with Final Import sheet
column_mapping = {
    'Effective Date': 'Rate - Valid from',
    'Expiry Date': 'Rate - Valid to',
    'Origin Port': 'Origin Port (name)',
    'Origin Port Code': 'Origin Port (code)',
    'Destination Port': 'Destination Port (name)',
    'Destination Port Code': 'Destination Port (code)',
    'Commodity Name': 'Commodity',
    'THC Used': 'THC Used',
    'Customer': 'Customer',
    'Contract Number': 'Contract Number',
    'Carrier': 'Carrier'
}

ocean_rates.rename(columns=column_mapping, inplace=True)

# Fill missing columns with NaN to match Final Import structure
missing_columns = final_import.columns.difference(ocean_rates.columns)
for col in missing_columns:
    ocean_rates[col] = np.nan


    
    
'''
convert nan into "NONE" to avoid blank cell, also can replace 'NONE' with 'None'. 
I wanted to keep all blank cell similar, so I used 'NONE' to match with "THC Used" column.

'''    
ocean_rates = ocean_rates.fillna("NONE")     

    
ocean_rates = ocean_rates[final_import.columns]




ocean_rates

,Rate - Valid from,Rate - Valid to,Origin Port (name),Origin Port (code),Destination Port (name),Destination Port (code),Customer,Carrier,Contract Number,Commodity,THC Used,"BAS, Currency","BAS, 20DC","BAS, 40DC","BAS, 40HC",…,"RHI, Currency","RHI, 20DC","RHI, 40DC","RHI, 40HC"
0,29-Apr-2011,12-May-2011,"Anqing, China",CNAQG,"Cochin, India",INCOK,AWESOME CUSTOMER INC.,XENETA,Super Secret Contract,FAK,NONE,USD,760 USD,NONE,NONE,NONE,USD,760 USD,NONE,NONE
1,29-Apr-2011,12-May-2011,"Anqing, China",CNAQG,"Cochin, India",INCOK,AWESOME CUSTOMER INC.,XENETA,Super Secret Contract,FAK,NONE,USD,0 USD,NONE,NONE,NONE,USD,0 USD,NONE,NONE
2,29-Apr-2011,12-May-2011,"Anqing, China",CNAQG,"Cochin, India",INCOK,AWESOME CUSTOMER INC.,XENETA,Super Secret Contract,FAK,NONE,USD,0 USD,NONE,NONE,NONE,USD,0 USD,NONE,NONE
3,29-Apr-2011,12-May-2011,"Anqing, China",CNAQG,"Cochin, India",INCOK,AWESOME CUSTOMER INC.,XENETA,Super Secret Contract,FAK,NONE,INR,10175 INR,NONE,NONE,NONE,INR,10175 INR,NONE,NONE
4,29-Apr-2011,12-May-2011,"Anqing, China",CNAQG,"Cochin, India",INCOK,AWESOME CUSTOMER INC.,XENETA,Super Secret Contract,FAK,NONE,USD,0 USD,NONE,NONE,NONE,USD,0 USD,NONE,NONE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41548,29-Apr-2011,12-May-2011,"Zhuhai, China",CNZUH,"Visakhapatnam, India",INVTZ,AWESOME CUSTOMER INC.,XENETA,Super Secret Contract,FAK,NONE,CNY,NONE,NONE,50 CNY,NONE,CNY,NONE,NONE,50 CNY
41549,29-Apr-2011,12-May-2011,"Zhuhai, China",CNZUH,"Visakhapatnam, India",INVTZ,AWESOME CUSTOMER INC.,XENETA,Super Secret Contract,FAK,NONE,INR,NONE,NONE,2600 INR,NONE,INR,NONE,NONE,2600 INR
41550,29-Apr-2011,12-May-2011,"Zhuhai, China",CNZUH,"Visakhapatnam, India",INVTZ,AWESOME CUSTOMER INC.,XENETA,Super Secret Contract,FAK,NONE,USD,NONE,NONE,24 USD,NONE,USD,NONE,NONE,24 USD
41551,29-Apr-2011,12-May-2011,"Zhuhai, China",CNZUH,"Visakhapatnam, India",INVTZ,AWESOME CUSTOMER INC.,XENETA,Super Secret Contract,FAK,NONE,CNY,NONE,NONE,890 CNY,NONE,CNY,NONE,NONE,890 CNY


In [16]:

# Save to Excel
output_path = "output.xlsx"
ocean_rates.to_excel(output_path, index=False)

print(f"file saved to {output_path}.")

file saved to output.xlsx.


In [17]:
output = pd.read_excel("output.xlsx")


output

,Rate - Valid from,Rate - Valid to,Origin Port (name),Origin Port (code),Destination Port (name),Destination Port (code),Customer,Carrier,Contract Number,Commodity,THC Used,"BAS, Currency","BAS, 20DC","BAS, 40DC","BAS, 40HC",…,"RHI, Currency","RHI, 20DC","RHI, 40DC","RHI, 40HC"
0,29-Apr-2011,12-May-2011,"Anqing, China",CNAQG,"Cochin, India",INCOK,AWESOME CUSTOMER INC.,XENETA,Super Secret Contract,FAK,NONE,USD,760 USD,NONE,NONE,NONE,USD,760 USD,NONE,NONE
1,29-Apr-2011,12-May-2011,"Anqing, China",CNAQG,"Cochin, India",INCOK,AWESOME CUSTOMER INC.,XENETA,Super Secret Contract,FAK,NONE,USD,0 USD,NONE,NONE,NONE,USD,0 USD,NONE,NONE
2,29-Apr-2011,12-May-2011,"Anqing, China",CNAQG,"Cochin, India",INCOK,AWESOME CUSTOMER INC.,XENETA,Super Secret Contract,FAK,NONE,USD,0 USD,NONE,NONE,NONE,USD,0 USD,NONE,NONE
3,29-Apr-2011,12-May-2011,"Anqing, China",CNAQG,"Cochin, India",INCOK,AWESOME CUSTOMER INC.,XENETA,Super Secret Contract,FAK,NONE,INR,10175 INR,NONE,NONE,NONE,INR,10175 INR,NONE,NONE
4,29-Apr-2011,12-May-2011,"Anqing, China",CNAQG,"Cochin, India",INCOK,AWESOME CUSTOMER INC.,XENETA,Super Secret Contract,FAK,NONE,USD,0 USD,NONE,NONE,NONE,USD,0 USD,NONE,NONE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41548,29-Apr-2011,12-May-2011,"Zhuhai, China",CNZUH,"Visakhapatnam, India",INVTZ,AWESOME CUSTOMER INC.,XENETA,Super Secret Contract,FAK,NONE,CNY,NONE,NONE,50 CNY,NONE,CNY,NONE,NONE,50 CNY
41549,29-Apr-2011,12-May-2011,"Zhuhai, China",CNZUH,"Visakhapatnam, India",INVTZ,AWESOME CUSTOMER INC.,XENETA,Super Secret Contract,FAK,NONE,INR,NONE,NONE,2600 INR,NONE,INR,NONE,NONE,2600 INR
41550,29-Apr-2011,12-May-2011,"Zhuhai, China",CNZUH,"Visakhapatnam, India",INVTZ,AWESOME CUSTOMER INC.,XENETA,Super Secret Contract,FAK,NONE,USD,NONE,NONE,24 USD,NONE,USD,NONE,NONE,24 USD
41551,29-Apr-2011,12-May-2011,"Zhuhai, China",CNZUH,"Visakhapatnam, India",INVTZ,AWESOME CUSTOMER INC.,XENETA,Super Secret Contract,FAK,NONE,CNY,NONE,NONE,890 CNY,NONE,CNY,NONE,NONE,890 CNY


In [18]:
print(output.columns)

final_import = pd.read_excel(excel_path, sheet_name="Final Import")
print(final_import.columns)


'''
##########Assumption############
Now we can see that the columns in the output.xlsx file match those in the "Final Import" sheet. 
Therefore, it can be assumed that the output data from the output.xlsx file meets the requirements of the "Final Import" sheet.
'''

Index(['Rate - Valid from', 'Rate - Valid to', 'Origin Port (name)',
       'Origin Port (code)', 'Destination Port (name)',
       'Destination Port (code)', 'Customer', 'Carrier', 'Contract Number',
       'Commodity', 'THC Used', 'BAS, Currency', 'BAS, 20DC', 'BAS, 40DC',
       'BAS, 40HC', '…', 'RHI, Currency', 'RHI, 20DC', 'RHI, 40DC',
       'RHI, 40HC'],
      dtype='object')
Index(['Rate - Valid from', 'Rate - Valid to', 'Origin Port (name)',
       'Origin Port (code)', 'Destination Port (name)',
       'Destination Port (code)', 'Customer', 'Carrier', 'Contract Number',
       'Commodity', 'THC Used', 'BAS, Currency', 'BAS, 20DC', 'BAS, 40DC',
       'BAS, 40HC', '…', 'RHI, Currency', 'RHI, 20DC', 'RHI, 40DC',
       'RHI, 40HC'],
      dtype='object')


'\n##########Assumption############\nNow we can see that the columns in the output.xlsx file match those in the "Final Import" sheet. \nTherefore, it can be assumed that the output data from the output.xlsx file meets the requirements of the "Final Import" sheet.\n'